In [1]:
import random
import mph
import numpy as np
import matplotlib.pyplot as plt
import os    
import time
from tqdm.notebook import tqdm
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'

path = 'D:/Josh/Fall 2021/ML-LWA/1dsim.mph'

scattering = [1,2,5,6,7,51,53,56,57]

In [2]:
def start_client(path):
    global model
    global pymodel
    global client
    client = mph.start(version = '5.3a')
    pymodel = client.load(path)
    model = pymodel.java
    
def save_model():
    model.save(path)
    
def generate_image(random_pattern):
    adjusted = [x - 15 for x in random_pattern]
    out = np.flip(np.array([1 if x in adjusted else 0 for x in np.arange(36)]))
    return out
    
def simulate_random():
    global start_count
    grids_of_interest = [random.randint(15,50) for i in range(0,random.randint(1,36))]
    this_scattering = scattering + grids_of_interest
    
    model.physics("emw").feature("sctr1").selection().set(this_scattering);
    model.physics("emw").feature("ffd1").feature("ffc1").selection().set(grids_of_interest);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()

    return grids_of_interest

In [ ]:
save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/1d/' + str(int(time.time()))
start_client(path)
goi = simulate_random()
array = generate_image(goi)

<h1>Please don't close out of this, very long simulation in progress! Using the computer is fine just don't close this window, thanks! -Josh</h1>

In [3]:
for instance in tqdm(np.arange(10)):
    img_list = []

    save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/1d/' + str(int(time.time()))

    start_client(path)
    start_time = time.time()
    for i in tqdm(np.arange(500)):
        goi = simulate_random()
        array = generate_image(goi)
        img_list.append(array)

    print(time.time() - start_time, 'seconds')

    with open(save_str + '.pkl', 'wb') as file:
        pickle.dump(img_list, file)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

18465.425374746323 seconds


  0%|          | 0/500 [00:00<?, ?it/s]

18622.494925498962 seconds


...

19179.156658887863 seconds


  0%|          | 0/500 [00:00<?, ?it/s]

19880.392385482788 seconds


  0%|          | 0/500 [00:00<?, ?it/s]

19463.81964802742 seconds


In [ ]:
model.save('D:/Josh/Fall 2021/ML-LWA/check1d2.mph')

In [ ]:
plt.imshow(array.reshape([36,1,1]))